<a href="https://colab.research.google.com/github/saishshinde15/Langchain_RAG_BOT/blob/main/RAG_Using_FreeResources_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00


In [3]:
%pip install -qU langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 3.9 MB/s eta 0:00:00


In [4]:
from langchain_community.document_loaders import PyPDFLoader

In [78]:
loader=PyPDFLoader('/content/HAI_AI-Index-Report-2024.pdf')

In [79]:
docs=loader.load()

In [11]:
%pip install -qU langchain-text-splitters

In [80]:
from langchain_text_splitters import CharacterTextSplitter

In [81]:
text_splitter= CharacterTextSplitter(chunk_size=1000,chunk_overlap=0,length_function=len)

In [82]:
docs=text_splitter.split_documents(docs)

In [83]:
len(docs)

502

In [101]:
import getpass

inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")


Enter your HF Inference API Key:

··········


In [102]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

In [103]:
embeddings = HuggingFaceInferenceAPIEmbeddings(api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2",)

In [106]:
query_result = embeddings.embed_query(docs[0].page_content)


In [107]:
from langchain_community.vectorstores import FAISS

In [108]:
db=FAISS.from_documents(docs,embeddings)

In [113]:
query="What is the report about?"
result=db.similarity_search(query,k=2)

In [131]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [137]:
import getpass
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass(
    "Enter your Hugging Face API key: "
)

Enter your Hugging Face API key: ··········


In [138]:
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [139]:

chat_model = ChatHuggingFace(llm=llm)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [140]:
from langchain_core.prompts import ChatPromptTemplate


In [158]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""


In [162]:
query='What is Machine Learning'

In [163]:
CHAT_PROMPT = ChatPromptTemplate.from_template(template)

chain = CHAT_PROMPT | chat_model

response=chain.invoke({"context": result, "question": query})


In [164]:
response.content

'Based on the context provided, the concept of Machine Learning is not explicitly mentioned. However, Machine Learning is a subfield of Artificial Intelligence (AI) that involves training algorithms on large datasets to enable them to make accurate predictions or decisions without being explicitly programmed. As Machine Learning is a critical aspect of AI, it is often discussed in the context of AI development and implementation. In the context of the AI Index Report 2024, which is a report focused on tracking the progress and'

# Other Method


In [ ]:
#from langchain.chains.combine_documents import create_stuff_documents_chain
#document_chain=create_stuff_documents_chain(llm,prompt)

In [143]:
#retriever=db.as_retriever()
#retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7884f256fb20>)

In [123]:
#from langchain.memory import ConversationBufferMemory
#memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)

In [144]:
#from langchain.chains import create_retrieval_chain
#chain=create_retrieval_chain(retriever,document_chain)

In [152]:
#response=chain.invoke({"input":"What is AI"})

In [ ]:
#response['answer']